# Chains

Chains é a feature principal do langchain que permite que se construa um pipeline com interações de código, usuário e modelo.

## Setup

In [24]:
from dotenv import find_dotenv, load_dotenv
from langchain_openai import ChatOpenAI, OpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

_ = load_dotenv(find_dotenv('.env'))

In [2]:
openai = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)

## Simples

In [4]:
prompt_template = PromptTemplate.from_template(
    'Descreva as tendências tecnológicas em {ano}'
)

runnable_sequence = prompt_template | openai
output = runnable_sequence.invoke({'ano': '2024'})

print(output)



1. Inteligência Artificial (IA) e Aprendizado de Máquina (Machine Learning): A IA e o Aprendizado de Máquina continuarão a ser tendências tecnológicas dominantes em 2024. Com o avanço da tecnologia, a IA será cada vez mais integrada em diversos setores, como saúde, transporte, finanças e educação. Além disso, o Aprendizado de Máquina permitirá que os sistemas se tornem mais inteligentes e autônomos, aprendendo e se adaptando com base em dados e experiências anteriores.

2. Internet das Coisas (IoT): A IoT também continuará a ser uma tendência tecnológica importante em 2024. Com a crescente conectividade entre dispositivos, a IoT permitirá que as pessoas controlem e monitorem suas casas, carros e outros objetos de forma remota. Além disso, a IoT será cada vez mais utilizada em ambientes industriais, permitindo a automação e otimização de processos.

3. Realidade Virtual (VR) e Realidade Aumentada (AR): A VR e a AR estão se tornando cada


## Sequencial

In [6]:
openai = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [13]:
# Base de cohecimento
loader = TextLoader('base_conhecimento_britadeira.txt')
documents = loader.load()
history = """Cliente: Minha britadeira não liga.
Chatbot: Você já verificou se a bateria está carregada e conectada corretamente?"""
question = "Já verifiquei e está carregada e conectada, mas ainda não liga"

In [20]:
context = '\n'.join(doc.page_content for doc in documents)

In [17]:
prompt_knowledge = PromptTemplate(
    input_variables=['context', 'question'],
    template="""Use o seguinte manual para responder à pergunta.
Responda apenas com base nas informações fornecidas.
Não forneça instruções de procedimentos já realizados.
Não utilize informações externas ao manual.
Manual: {context}
Pergunta: {question}"""
)

prompt_history = PromptTemplate(
    input_variables=['history', 'question'],
    template="""Use o histórico da conversa para responder à pergunta.
Responda apenas com base nas informações fornecidas.
Não forneça instruções de procedimentos já realizados.
Não utilize informações externas ao manual.
Histórico: {history}
Pergunta: {question}"""
)

prompt_end = PromptTemplate(
    input_variables=['response_knownledge', 'response_history'],
    template="""Combine as seguintas respostas para responder à pergunta.
Responda apenas com base nas informações fornecidas.
Não forneça instruções de procedimentos já realizados.
Resposta a partir da base de conhecimento: {response_knowledge}
Resposta a partir do histórico de conversa: {response_history}"""
)

In [18]:
chain_knowledge = prompt_knowledge | openai
chain_history = prompt_history | openai
chain_end = prompt_end | openai

In [22]:
response_knowledge = chain_knowledge.invoke({'context': context, 'question': question})
response_history = chain_history.invoke({'history': history, 'question': question})
response_end = chain_end.invoke({'response_knowledge': response_knowledge, 'response_history': response_history})

print(response_end.content)

Verifique se o interruptor liga/desliga está na posição correta e se há algum fusível queimado ou se o cabo de alimentação está danificado. Se o problema persistir, contate o suporte técnico pelo 0800 555 5555.


## Router

In [23]:
openai = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)

In [26]:
chain = (
    PromptTemplate.from_template(
        """Classifique a pergunta do usuário em uma das seguintes estratégias:
- Assuntos financeiros
- Suporte técnico
- Atualização de cadastro
- Outras informações

Pergunta: {query}
Classificação:"""
    )
    | openai
    | StrOutputParser()
)

print(chain)

first=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='Classifique a pergunta do usuário em uma das seguintes estratégias:\n- Assuntos financeiros\n- Suporte técnico\n- Atualização de cadastro\n- Outras informações\n\nPergunta: {query}\nClassificação:') middle=[OpenAI(client=<openai.resources.completions.Completions object at 0x7fb36cfecb90>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7fb36cfec910>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))] last=StrOutputParser()


In [27]:
financial_chain = PromptTemplate.from_template(
    """Você é um especialista financeiro
Sempre responda às perguntas com "Bem vindo ao Suporte Financeiro".
Responda à pergunta do usuário:
Pergunta: {query}
Resposta:"""
) | openai
tech_chain = PromptTemplate.from_template(
    """Você é um especialista em suporte técnico 
Sempre responda às perguntas com "Bem vindo ao Suporte Técnico".
Responda à pergunta do usuário:
Pergunta: {query}
Resposta:"""
) | openai
registration_chain = PromptTemplate.from_template(
    """Você é um representante de atendimento ao cliente
Sempre responda às perguntas com "Bem vindo ao Suporte de Cadastro".
Responda à pergunta do usuário:
Pergunta: {query}
Resposta:"""
) | openai
other_chain = PromptTemplate.from_template(
    """Você é um assistente de informações gerais
Sempre responda às perguntas com "Bem vindo ao Suporte Geral".
Responda à pergunta do usuário:
Pergunta: {query}
Resposta:"""
) | openai

In [35]:
def route(info):
    topic = info['topic'].lower()
    match topic:
        case x if 'financeiro' in x:
            return financial_chain
        case x if 'técnico' in x:
            return tech_chain
        case x if 'atualização' in x:
            return registration_chain
        case x if 'cadastro' in x:
            return registration_chain
        case True:
            return other_chain

In [37]:
QUERY = 'Como faço para redefinir minha senha?'

In [38]:
classification = chain.invoke({'query': QUERY})
print(classification)

 Suporte técnico


In [39]:
response_chain = route({'topic': classification})
print(response_chain)

first=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='Você é um especialista em suporte técnico \nSempre responda às perguntas com "Bem vindo ao Suporte Técnico".\nResponda à pergunta do usuário:\nPergunta: {query}\nResposta:') middle=[] last=OpenAI(client=<openai.resources.completions.Completions object at 0x7fb36cfecb90>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7fb36cfec910>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))


In [41]:
response = response_chain.invoke({'query': QUERY})
print(response)

 Bem vindo ao Suporte Técnico. Para redefinir sua senha, você pode seguir as instruções fornecidas pelo sistema ou entrar em contato conosco para obter assistência adicional. Faremos o possível para ajudá-lo a recuperar o acesso à sua conta.
